In [1]:
import pandas as pd
import numpy as np

print("Pandas Version: " + pd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.2.4
Numpy Version: 1.18.5


In [2]:
trips = pd.read_csv("../data/clean/trips.csv", parse_dates=["date_from", "date_until"])
trips.head(3)

,bike_id,user_id,date_from,date_until,start_station_name,start_station_id,end_station_name,end_station_id,booked_via,duration_in_min,distance_in_km,speed_in_kmh
0,143517,A821059B555C7764A2FF801180874A2FCB326222,2014-01-01 00:34:54,2014-01-01 00:50:14,U-Bahn Baumwall,214170,Mönckebergstraße / Rosenstraße,131880,iPhone SRH,16,1.293661,4.851229
1,119830,1EBC930DB407ACEAE2FDE23A6CA40492EA3DFBB2,2014-01-01 01:39:55,2014-01-01 01:57:27,Bahnhof Altona Ost/Max-Brauer-Allee,131646,Schulterblatt/Eifflerstraße,131648,Android SRH,18,2.032271,6.774236
2,143501,7AD2C1B70137479062A6DD73815835986677BB2D,2014-01-01 01:40:20,2014-01-01 01:53:09,Weidestraße/Biedermannplatz,211922,Jarrestraße / Rambatzweg,138376,Techniker HH_119 (-2334-),13,0.954178,4.403899


In [3]:
user_groups = trips.groupby(["user_id"])

In [4]:
last_moment = trips.date_until.max()

In [5]:
user_trip_dates = trips.groupby(["user_id"], as_index=False).agg({"date_from": [np.min, np.max]})
user_trip_dates.columns = pd.Index(["user_id", "date_from_min", "date_from_max"])
user_trip_dates.set_index("user_id", inplace=True)

In [6]:
trips_count = user_groups.size().to_frame()
trips_count.rename(columns = {0: "trips_count"}, inplace=True)

In [7]:
active_period_in_days = user_groups.apply(
    lambda x: (x.date_from.max() - x.date_from.min()).days
).to_frame()
active_period_in_days.rename(columns = {0: "active_period_in_days"}, inplace=True)

In [8]:
days_since_first_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.min()).days
).to_frame()
days_since_first_activity.rename(columns = {0: "days_since_first_activity"}, inplace=True)

In [9]:
days_since_last_activity = user_groups.apply(
    lambda x: (last_moment - x.date_from.max()).days
).to_frame()
days_since_last_activity.rename(columns = {0: "days_since_last_activity"}, inplace=True)

In [10]:
mean_duration_in_min = user_groups.apply(
    lambda x: x.duration_in_min.mean()
).to_frame()
mean_duration_in_min.rename(columns = {0: "mean_duration_in_min"}, inplace=True)

In [11]:
mean_distance_in_km = user_groups.apply(
    lambda x: x.distance_in_km.mean()
).to_frame()
mean_distance_in_km.rename(columns = {0: "mean_distance_in_km"}, inplace=True)

In [12]:
mean_speed_in_kmh = user_groups.apply(
    lambda x: x.speed_in_kmh.mean()
).to_frame()
mean_speed_in_kmh.rename(columns = {0: "mean_speed_in_kmh"}, inplace=True)

In [13]:
def calculate_mean_time_between_trips_in_hours(group):
    sorted_group = group.sort_values(by="date_from").reset_index()
    total_in_seconds = 0
    for index, row in sorted_group.iterrows():
        if (index != sorted_group.index[-1]):
            next_trip = sorted_group.iloc[index+1]
            delta = next_trip.date_from - row.date_from
            total_in_seconds = total_in_seconds + delta.seconds
    total_in_hours = total_in_seconds / 3600
    return total_in_hours/len(sorted_group)

# TODO can this be faster?
#mean_time_between_trips_in_hours = user_groups.apply(calculate_mean_time_between_trips_in_hours).to_frame()
#mean_time_between_trips_in_hours.rename(columns = {0: "mean_time_between_trips_in_hours"}, inplace=True)

In [14]:
users = pd.merge(user_trip_dates, trips_count, left_index=True, right_index=True)
users = pd.merge(users, active_period_in_days, left_index=True, right_index=True)
users = pd.merge(users, days_since_first_activity, left_index=True, right_index=True)
users = pd.merge(users, days_since_last_activity, left_index=True, right_index=True)
users = pd.merge(users, mean_duration_in_min, left_index=True, right_index=True)
users = pd.merge(users, mean_distance_in_km, left_index=True, right_index=True)
users = pd.merge(users, mean_speed_in_kmh, left_index=True, right_index=True)
#users = pd.merge(users, mean_time_between_trips_in_hours, left_index=True, right_index=True)

In [15]:
# Labelling users as 'casual' or 'regular' based on activity patterns.

REGULAR_USER_ACTIVE_DAYS_MIN = 28
REGULAR_USER_THRESHOLD = 4

def label_user_type(user):
    if (
        (user.active_period_in_days / user.trips_count) <= REGULAR_USER_THRESHOLD
    ) & (
        user.active_period_in_days >= REGULAR_USER_ACTIVE_DAYS_MIN
    ):
        return "regular"
    else:
        return "casual"

users["type"] = users.apply(label_user_type, axis=1)

In [16]:
users.sample(5)

,date_from_min,date_from_max,trips_count,active_period_in_days,days_since_first_activity,days_since_last_activity,mean_duration_in_min,mean_distance_in_km,mean_speed_in_kmh,type
user_id,,,,,,,,,,
233591E4325DF4A38D5FC78017DA9EEE889F802E,2017-04-06 18:24:21,2017-05-06 16:01:16,4,29,39,9,19.500000,2.369520,7.304323,casual
74D0C8B4B011AFB5D1DBE96EFE6879488155ADF7,2014-01-25 23:29:10,2016-10-31 17:35:12,11,1009,1206,196,16.000000,1.662806,6.359780,casual
7226BA8678EE8875FF6533718BEC42372B4A616B,2014-03-12 07:03:08,2016-06-30 16:49:11,4,841,1161,319,6.500000,0.689927,6.669291,casual
C9CC2FC2C305B69BC03441DE07FBBF2F3F5A2E2E,2014-03-24 17:27:38,2014-03-28 11:11:28,6,3,1148,1144,8.000000,1.169435,8.863572,casual
917116898897728EE8D54DCB00668B0E6194A3E9,2014-05-13 21:07:32,2015-05-24 15:23:58,41,375,1098,722,14.073171,0.999074,5.043819,casual


In [17]:
users.to_csv("../data/clean/users.csv", index=True)